In [1]:
import sympy as sp
import jax.numpy as np
import matplotlib.pyplot as plt
from util.dotdict import DotDict
from eq5 import config
from main import run

In [2]:
network, best = run(config)

22:01:31.198 [INFO] Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
22:01:31.199 [INFO] Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Interpreter Host
22:01:31.199 [INFO] Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
22:01:31.554 [INFO] Constructed symbolic model
22:01:32.325 [INFO] Constructed loss equation
22:01:33.800 [INFO] Constructed JAXified model
22:01:36.184 [INFO] Epoch: 1, Loss: 5.8141021728515625
22:01:36.350 [INFO] Epoch: 10, Loss: 3.398131847381592
22:01:36.531 [INFO] Epoch: 20, Loss: 1.9249848127365112
22:01:36.703 [INFO] Epoch: 30, Loss: 1.1127853393554688
22:01:36.885 [INFO] Epoch: 40, Loss: 0.6525678634643555
22:01:37.580 [INFO] Epoch: 50, Loss: 0.3869858682155609
22:01:37.231 [INFO] Epoch: 60, Loss: 0.23175352811813354
22:01:37.401 [INFO] Epoch: 70, Loss: 0.14105764031410217
22:01:37.574 [INF

Nothing more to prune!


$\displaystyle - 0.0191672672761928 t x - 0.0149035214030087 x - 0.0240293726096177$

In [14]:
float(best.loss)

0.00595303438603878

In [8]:
y_prediction_best = best.model_y.subs(zip(best.alphas, best.W))
y_prediction_best

-0.0191672672761928*t*x - 0.0149035214030087*x - 0.0240293726096177

In [9]:
y_prediction_re = y_prediction_best

newsymbols = DotDict()
for name, varinfo in config.vars.items():
  if varinfo.symbol is not None:
      newsymbols[name] = sp.symbols(varinfo.symbol)
      y_prediction_re = y_prediction_re.subs(name, newsymbols[name])

y_prediction_re

-0.0191672672761928*\theta_0*\xi - 0.0149035214030087*\xi - 0.0240293726096177

In [10]:
from jax import vmap
x = network.symbols.x
t = network.symbols.t
b = network.symbols.b
y_pred_fn = sp.lambdify([t, x, b], y_prediction_best)
# y_real_fn = sp.lambdify(..?)

In [13]:
loss_sym = t * sp.diff(y_prediction_best, x) + b * y_prediction_best * (1 - y_prediction_best)

sp.integrate(loss_sym**2, (t, 0.01, 1), (x, -20, 20), (b, 0.01, 1))

1.57620272687226

In [ ]:
# x_try = np.linspace(0, 1, 50)

# fig, axs = plt.subplots(1, len(k_tries))
# fig.set_size_inches(len(k_tries)*3.2, 3)

# for ax, k_try in zip(axs, k_tries):
#   ax.set_title(f'k={round(float(k_try), 3)}')
#   ax.plot(x_try, y_pred_fn(x_try, k_try), label=f'prediction')
#   # ax.plot(x_try, y_real_fn(x_try, k_try), label=f'actual')
#   ax.legend()
# plt.show()